In [1]:
cd ../../

/data/mayu-ot/Experiments/neurocomp_vgp


In [18]:
import numpy as np
from dataset import Dataset, conv_f
from chainer.dataset.convert import to_device

In [19]:
def group_phrases(A):
    A = A.copy()
    groups = []
    i = 0
    while(i<len(A)):
        g = np.where(A[i]==1)[0]
        A[:, g] = -1
        groups.append(g)
        cand = np.where(A[i]==0)[0]
        if len(cand)==0:
            break
        i = cand[cand>i][0]
        
    label = np.zeros((len(A),), dtype=np.uint8)
    for i, g in enumerate(groups):
        label[g] = i
    return label

In [106]:
import pandas as pd
from itertools import combinations
from scipy.spatial.distance import squareform
import numpy as np
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import adjusted_rand_score

df = pd.read_csv('models/tpami/SNN+DDPN/20191213-151225/pred_val.csv')
clutter=.5
damping=.5

ad_rands = []

In [108]:
img_id = df.image.sample().values[0]
sub_df = df[df.image==img_id]
phrases = pd.unique(sub_df[['phrase1', 'phrase2']].values.ravel())
n = len(phrases)
scores = []
t_label = []

for i_1, i_2 in combinations(range(n), 2):
    p1 = phrases[i_1]
    p2 = phrases[i_2]
    
    rows = sub_df[(sub_df.phrase1==p1)&(sub_df.phrase2==p2)]
    
    if len(rows):
        rows = sub_df[(sub_df.phrase1==p2)&(sub_df.phrase2==p1)]

    s = rows.pred.values[0] if len(rows) else 0
    t = rows.label.values[0] if len(rows) else 0
    scores.append(s)
    t_label.append(t)

A = squareform(scores)
T = squareform(t_label) + np.eye(n)

In [109]:
%time
pref = np.percentile(scores, clutter)
af = AffinityPropagation(preference=pref, affinity='precomputed',damping=damping)
label = af.fit_predict(A)
gt_label = group_phrases(T)
adr_s = adjusted_rand_score(gt_label, label)
ad_rands.append(adr_s)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 12.9 µs


In [38]:
def conv_f(batch, device=None):
    p1 = [to_device(device, np.asarray(b[0]).astype('i')) for b in batch]
    p2 = [to_device(device, np.asarray(b[1]).astype('i')) for b in batch]
    v1 = [b[2] for b in batch]
    v2 = [b[3] for b in batch]
    l = [b[4] for b in batch]
    
    v1 = np.vstack(v1).astype('f')
    v2 = np.vstack(v2).astype('f')
    l = np.asarray(l).astype('i')
    
    v1 = to_device(device, v1)
    v2 = to_device(device, v2)
    l = to_device(device, l)
    
    return p1, p2, v1, v2, l

def conv_f2(batch, device=None):
    p1 = []
    p2 = []
    v1 = []
    v2 = []
    l = []
    for b in batch:
        p1.append(to_device(device, np.asarray(b[0]).astype('i')))
        p2.append(to_device(device, np.asarray(b[1]).astype('i')))
        v1.append(b[2])
        v2.append(b[3])
        l.append(b[4])
        
    v1 = np.vstack(v1).astype('f')
    v2 = np.vstack(v2).astype('f')
    l = np.asarray(l).astype('i')
    
    v1 = to_device(device, v1)
    v2 = to_device(device, v2)
    l = to_device(device, l)
    
    return p1, p2, v1, v2, l

def sample():
    i = np.random.randint(len(train), size=100)
    batch = train[i]
    batch = conv_f(batch, device=3)
    return batch
def sample2():
    i = np.random.randint(len(train), size=100)
    batch = train[i]
    _ = conv_f2(batch, device=3)

In [39]:
%timeit sample()

14.6 ms ± 549 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
%timeit sample2()

14 ms ± 91.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
import pstats
from pstats import SortKey

p = pstats.Stats('profile.stats')
p.strip_dirs().sort_stats(-1).print_stats()

Fri Dec 13 12:42:54 2019    profile.stats

         9100917 function calls (9057639 primitive calls) in 36.053 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <decorator-gen-0>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-10>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-1>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-2>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-3>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-4>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-5>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-6>:1(<module>)
        1    0.000    0.000    0.000    0.000 <decorator-gen-7>:1(<module>)
        3    0.000    0.000    0.000    0.000 <decorator-gen-7>:1(_held_figure)
        1    

In [4]:
p.sort_stats(SortKey.CUMULATIVE).print_stats(30)

Fri Dec 13 12:42:54 2019    profile.stats

         9100917 function calls (9057639 primitive calls) in 36.053 seconds

   Ordered by: cumulative time
   List reduced from 8002 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   1936/1    0.023    0.000   36.056   36.056 {built-in method builtins.exec}
        1    0.000    0.000   36.056   36.056 train_model.py:1(<module>)
        1    0.000    0.000   34.460   34.460 train_model.py:109(main)
        1    0.002    0.002   34.459   34.459 train_model.py:24(train)
        1    0.001    0.001   23.789   23.789 trainer.py:293(run)
     14/7    0.000    0.000   13.576    1.939 npz.py:84(save_npz)
        7    0.000    0.000   13.565    1.938 npyio.py:636(savez_compressed)
        7    0.003    0.000   13.565    1.938 npyio.py:701(_savez)
      329    0.005    0.000   13.525    0.041 format.py:574(write_array)
      987    0.003    0.000   13.369    0.014 zipfile.py:1089(write)
      987

In [43]:
from chainer import functions as F

In [54]:
n = 2000
W = np.random.random((n, 300))
X = [np.random.randint(n, size=np.random.randint(10)+1) for _ in range(5)]

%time emb = [F.embed_id(x, W) for x in X]

CPU times: user 514 µs, sys: 90 µs, total: 604 µs
Wall time: 609 µs


In [60]:
n = 2000
W = np.random.random((n, 300))
X = [np.random.randint(n, size=np.random.randint(10)+1) for _ in range(5)]

def embed_1(X, W):
    emb = [F.embed_id(x, W) for x in X]
    return emb

def embed_2(X, W):
    indices = [len(x) for x in X]
    X = np.hstack(X)
    emb = F.embed_id(X, W)
    emb = F.split_axis(emb, indices, axis=0)
    return emb

%time _ = embed_1(X, W)

CPU times: user 550 µs, sys: 97 µs, total: 647 µs
Wall time: 652 µs


In [61]:
%time _ = embed_2(X, W)

CPU times: user 1.1 ms, sys: 0 ns, total: 1.1 ms
Wall time: 1.08 ms
